# Submission

In [11]:
import os, joblib, torch
os.chdir('/local/disk4/chrenx/fog-challenge')
print(os.getcwd())
device = "cuda:0"

/local/disk4/chrenx/fog-challenge


In [12]:
def evaluation_metrics(output, gt):
    """Generate precision, recall, and f1 score.

    Args:
        output: (B, window, 1)   # prob class
        gt (inference):   (B, window, 3)   # one hot
    """
    # Convert the model output probabilities to class predictions
    pred = torch.round(output)  # (B, window, 1)

    # Extract the first two classes from the ground truth
    real = torch.argmax(gt[:, :, :2], dim=-1, keepdim=True)  # (B, window, 1)

    # Create a mask to ignore the positions where the ground truth class is 2
    mask = (gt[:, :, 2] != 1).unsqueeze(-1)  # (B, window, 1)

    # Apply the mask to the predictions and ground truth
    pred = (pred * mask.float()).squeeze() # (B, window)
    real = (real * mask.float()).squeeze() # (B, window)
    

    # Calculate true positives, false positives, and false negatives
    tp = ((pred == 1) & (real == 1)).float().sum()
    fp = ((pred == 1) & (real == 0)).float().sum()
    fn = ((pred == 0) & (real == 1)).float().sum()

    # Calculate precision, recall, and F1 score
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    
    return precision, recall, f1


In [13]:
test_dpath = 'data/rectified_data/kaggle_pd_data/test_kaggle_pd_data_window6976_lab_randomaug.p'
test_data = joblib.load(test_dpath)
test_data.keys(), test_data[0].keys()

(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
 dict_keys(['series_name', 'ori_series_name', 'start_t_idx', 'end_t_idx', 'model_input', 'gt', 'inference_gt']))

In [14]:
import os, shutil

for idx, element in test_data.items():
    dpath = os.path.join('data/rectified_data/kaggle_pd_data/tdcsfog', element['ori_series_name'][:-8]+'.csv')
    shutil.copy2(dpath, 'submission/unet_1/test_data')

In [15]:
model_input = torch.stack([test_data[i]['model_input'] for i in range(len(test_data.keys()))])
gt = torch.stack([test_data[i]['gt'] for i in range(len(test_data.keys()))])
model_input = model_input.to(device)
gt = gt.to(device)
model_input.shape, gt.shape

(torch.Size([25, 6976, 3]), torch.Size([25, 6976, 3]))

In [16]:
from submission.unet_1.codes.unet_v3 import UNet
weights_path = "submission/unet_1/weights/best_model_f1_3150.pt"
model = UNet(3)
model = model.to(device)
model.load_state_dict(torch.load(weights_path)['model']), model

(<All keys matched successfully>,
 UNet(
   (relu): ReLU()
   (sigmoid): Sigmoid()
   (e11): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=same)
   (e12): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=same)
   (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (e21): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=same)
   (e22): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=same)
   (maxpool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (e31): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
   (e32): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=same)
   (maxpool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (e41): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=same)
   (e42): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same)
   (maxpool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mod

#### Count num of parameters

In [6]:
def count_model_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad) 

count_model_parameters(model)

5784897

### Statistics

In [19]:
model.eval()
with torch.no_grad():
    test_input = torch.permute(model_input, (0,2,1)) # (B, num_feats, window)
    test_pred = model(test_input) # (B, 1, window)
    test_pred = torch.permute(test_pred, (0, 2, 1)) # (B, window, 1)
    prec, recall, f1 = evaluation_metrics(test_pred, gt)
prec.item(), recall, f1

(0.8476548790931702,
 tensor(0.8808, device='cuda:0'),
 tensor(0.8639, device='cuda:0'))

### FoG Model Output Data

In [9]:

model.eval()
with torch.no_grad():
    test_input = torch.permute(model_input, (0,2,1)) # (B, num_feats, window)
    test_pred = model(test_input) # (B, 1, window)
    output = torch.permute(test_pred, (0, 2, 1)) # (25, window, 1)
    

pred = torch.round(output)  # (B, window, 1)

# Extract the first two classes from the ground truth
real = torch.argmax(gt[:, :, :2], dim=-1, keepdim=True)  # (B, window, 1)

# Create a mask to ignore the positions where the ground truth class is 2
mask = (gt[:, :, 2] != 1).unsqueeze(-1)  # (B, window, 1)

# Apply the mask to the predictions and ground truth
output = (pred * mask.float()).squeeze() # (B, window)

output.shape

torch.Size([25, 6976])

In [10]:
import numpy as np
grouped_data = {}
for i in range(output.shape[0]):
    sample = test_data[i]
    ori_series_name = sample['ori_series_name']
    
    a = output[i].cpu().numpy()
    a[a == 0.0] = 0
    a[a == 1.0] = 1
    a[a == 2.0] = 2
    
    if ori_series_name not in grouped_data:
        grouped_data[ori_series_name] = a
    else:
        
        grouped_data[ori_series_name] = np.concatenate((grouped_data[ori_series_name], 
                                                        a))
print(len(grouped_data.keys()))
for key, value in grouped_data.items():
    print(f"sample: {key.split('_')[1]}, length: {value.size}")

16
sample: 808, length: 6976
sample: 60, length: 6976
sample: 703, length: 13952
sample: 289, length: 13952
sample: 698, length: 13952
sample: 127, length: 6976
sample: 721, length: 6976
sample: 354, length: 6976
sample: 654, length: 6976
sample: 285, length: 6976
sample: 443, length: 6976
sample: 738, length: 48832
sample: 122, length: 6976
sample: 481, length: 6976
sample: 696, length: 6976
sample: 223, length: 6976


In [22]:
import pandas as pd
# Create a DataFrame to hold the merged data
data = {f"ModelOutput_Trial{key.split('_')[1]}": value.astype(int).tolist() for key, value in grouped_data.items()}

max_len = max(len(v) for v in data.values())

for key in data:
    data[key] += [None] * (max_len - len(data[key]))
    
for key in data:
    for i in range(len(data[key])):
        if data[key][i] is not None:
            data[key][i] = int(data[key][i])


# with open('submission/RenHaocheng_FoG_Ground_Truth_Data.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     # Write the header
#     writer.writerow(data.keys())
#     # Write the rows
#     writer.writerows(zip(*data.values()))
    
# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('submission/RenHaocheng_FoG_Model_Output_Data.csv', index=False)
df = pd.read_csv('submission/RenHaocheng_FoG_Model_Output_Data.csv')


### FoG GT Data

In [31]:
import pandas as pd
import numpy as np

# Function to convert one-hot encoding to a single value
def one_hot_to_label(one_hot):
    a = torch.argmax(one_hot[:, :], dim=-1)
    a[a == 2] = 0
    return a

# Group samples by 'ori_series_name'
grouped_data = {}

for i in range(len(test_data.keys())):
    sample = test_data[i]
    ori_series_name = sample['ori_series_name']
    gt_label = one_hot_to_label(sample['gt'])
    
    if ori_series_name not in grouped_data:
        grouped_data[ori_series_name] = gt_label.cpu().numpy()
    else:
        grouped_data[ori_series_name] = np.concatenate((grouped_data[ori_series_name], 
                                                        gt_label.cpu().numpy()))

print(len(grouped_data.keys()))
for key, value in grouped_data.items():
    print(f"sample: {key.split('_')[1]}, length: {value.size}")


16
sample: 808, length: 6976
sample: 60, length: 6976
sample: 703, length: 13952
sample: 289, length: 13952
sample: 698, length: 13952
sample: 127, length: 6976
sample: 721, length: 6976
sample: 354, length: 6976
sample: 654, length: 6976
sample: 285, length: 6976
sample: 443, length: 6976
sample: 738, length: 48832
sample: 122, length: 6976
sample: 481, length: 6976
sample: 696, length: 6976
sample: 223, length: 6976


In [32]:
import csv
# Create a DataFrame to hold the merged data
data = {f"GroundTruth_Trial{key.split('_')[1]}": value.astype(int).tolist() for key, value in grouped_data.items()}

max_len = max(len(v) for v in data.values())

for key in data:
    data[key] += [None] * (max_len - len(data[key]))

# with open('submission/RenHaocheng_FoG_Ground_Truth_Data.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     # Write the header
#     writer.writerow(data.keys())
#     # Write the rows
#     writer.writerows(zip(*data.values()))
    
# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('submission/RenHaocheng_FoG_Ground_Truth_Data.csv', index=False)
    
# with open('submission/RenHaocheng_FoG_Ground_Truth_Data.csv', 'w', newline='') as csvfile:
#     for key, value in data.items():
#         df = pd.DataFrame({key: value})
#         df.to_csv(csvfile, index=False, header=True)